In [1]:
import pandas as pd
import numpy as np
import pyodbc
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from datetime import date
import tkinter as tk
from tkinter.filedialog import askopenfilename
import sqlite3

In [2]:
conn = sqlite3.connect('../../InSyncConnection/Database/InSyncClinical.db')

In [3]:
import sys
import logging
path = "..\..\InSyncConnection\Code\clinical_log.txt"
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("CFTSS-TreatmentPlanReport")

# Notes
- Make sure that due dates are not calculated by time
- If there are two initial treatment plans go by the second one.
- Number of patients missing intial treatment plan should check if patient was seen recently rather than admitted

In [4]:
try:
    sql='''
    SELECT 
        ptPatient_Activity.*,
        MRNNumber,
        LastName || ', ' || FirstName as patient_fullName,
        DOB,
        FacilityName AS 'Primary Facility'
    FROM 
        ptPatient_Activity
        LEFT JOIN emr_PatientDetails ON (ptPatient_Activity.PatientID=emr_PatientDetails.PatientID)
        LEFT JOIN emr_FacilitiesDetails ON (emr_PatientDetails._PrimaryFacility = emr_FacilitiesDetails.FacilityId)
    '''
    patientDetails = pd.read_sql(sql, conn)
    patientDetails = patientDetails.replace("", np.nan)
    patientDetails['DOB'] =pd.to_datetime(patientDetails['DOB'])
    logger.info(f"Successfully queried ptPatient_Activity.")
except Exception as e:
    logger.error(f"Failed to query ptPatient_Activity.", exc_info=True) 

In [5]:
try:
    sql='''
    SELECT 
        tblEncounterSummary.PatientId as PatientID,
        tblEncounterSummary.EncounterID, 
        tblResourceDetails.FullName as ProviderName,
        tblEncounter.EndDate,
        VisitDateTime as EncounterDate,
        EncounterEndDateTimeToolTip,
        EncounterStartDateTimeToolTip,
        FacilityName,
        tblEncounterType.EncounterType,
        EncounterOpenClosed,
        ProgramName,
        closedByLastName || ', ' || closedByFirstName AS closedByFullName
    FROM 
        tblEncounterSummary
        LEFT JOIN emr_FacilitiesDetails ON (tblEncounterSummary.FacilityId = emr_FacilitiesDetails.FacilityId)
        LEFT JOIN tblEncounterEndDetails ON (tblEncounterSummary.EncounterID = tblEncounterEndDetails.EncounterID)
        INNER JOIN tblEncounter ON (tblEncounterSummary.EncounterID = tblEncounter.EncounterID)
        LEFT JOIN tblEncounterType ON (tblEncounterSummary.EncounterTypeID = tblEncounterType.EncounterTypeID)
        LEFT JOIN tblResourceDetails ON (tblEncounterSummary.ProviderId = tblResourceDetails.ResourceId)
    WHERE (tblEncounterType.EncounterType LIKE "%treatment%"
        OR tblEncounterType.EncounterType LIKE "%transfer%"
        OR tblEncounterType.EncounterType LIKE "%admission%"
        OR tblEncounterType.EncounterType LIKE "%discharge%")
        AND tblEncounterType.EncounterType LIKE "%CFTSS%"
    '''
    treatmentPlanEncounters = pd.read_sql(sql, conn)
    treatmentPlanEncounters = treatmentPlanEncounters.replace("", np.nan)
    treatmentPlanEncounters['EncounterDate'] = pd.to_datetime(treatmentPlanEncounters['EncounterDate'])
    treatmentPlanEncounters['EndDate'] = pd.to_datetime(treatmentPlanEncounters['EndDate'])
    # Remove the word time: from column
    treatmentPlanEncounters['EncounterStartDateTimeToolTip'] = treatmentPlanEncounters['EncounterStartDateTimeToolTip'].str.replace("Time: ", "")
    treatmentPlanEncounters['EncounterEndDateTimeToolTip'] = treatmentPlanEncounters['EncounterEndDateTimeToolTip'].str.replace("Time: ", "")

    treatmentPlanEncounters['EncounterStartDateTimeToolTip'] = pd.to_datetime(treatmentPlanEncounters['EncounterStartDateTimeToolTip'])
    treatmentPlanEncounters['EncounterEndDateTimeToolTip'] = pd.to_datetime(treatmentPlanEncounters['EncounterEndDateTimeToolTip'])
    logger.info(f"Successfully queried treatment plan related info from tblEncounterSummary.")
except Exception as e:
    logger.error(f"Failed to query treatment plan related info from tblEncounterSummary.", exc_info=True) 

## Check Patients for Admission
- For CFTSS a patient is classified as admitted if they had a billable session within 

In [6]:
try:
    sql = f'''
    SELECT
    PatientId as PatientID,
    VisitDateTime
    FROM 
        tblEncounterSummary
    LEFT JOIN
        tblEncounterType ON tblEncounterType.EncounterTypeID = tblEncounterSummary.EncounterTypeID
    WHERE 
        ISBILLABLE LIKE "%TRUE%"
        AND (tblEncounterType.EncounterType LIKE "%CPST%"
        OR tblEncounterType.EncounterType LIKE "%PSR%"
        OR tblEncounterType.EncounterType LIKE "%OLP%")
        AND NOT tblEncounterType.EncounterType LIKE "%Licensed Evaluation%"
    '''
    acceptedPatients = pd.read_sql(sql,conn)
    acceptedPatients['VisitDateTime'] = pd.to_datetime(acceptedPatients['VisitDateTime'])
    acceptedPatients.sort_values('VisitDateTime', ascending=False, inplace=True)
    acceptedPatients.drop_duplicates('PatientID', keep='first', inplace=True)
    acceptedPatients['Admitted']="True"
    treatmentPlanEncounters= treatmentPlanEncounters.merge(acceptedPatients[['PatientID', 'Admitted']], on='PatientID', how='left')
    treatmentPlanEncounters['Admitted'].fillna("False", inplace=True)
    logger.info(f"Successfully queried admitted patients. Patient must have either an encounter type of psychotherapy or an initial eval 2.")
except Exception as e:
    logger.error(f"Failed to query admitted patients. Patient must have either an encounter type of psychotherapy or an initial eval 2.", exc_info=True) 

In [7]:
def find_missing_initial_treatment_plan(temp_df):
    # if patient does not have initial treatment plan 
    # and first treatment plan review date is more than 6 months past our InSync onboarding 
    # and patient was admitted, 
    # return true otherwise return false
    if "CFTSS Initial Treatment Plan" not in temp_df['EncounterType'].unique() and str(temp_df[temp_df['Summarized Encounter Type'] == 'Treatment Plan']['EncounterDate'].first_valid_index()) != 'None' and str(temp_df[temp_df['Admitted'] == 'True']['Admitted'].first_valid_index()) != 'None':
        if temp_df.loc[temp_df[temp_df['Summarized Encounter Type'] == 'Treatment Plan']['EncounterDate'].first_valid_index(), 'EncounterDate'] > pd.Timestamp('1/20/2022'):
            return True
        else:
            return False
    else:
        return False
    
def getDischargeInfo(is_ActiveInInSync, is_lastEncounterDischarge):
    if is_ActiveInInSync and is_lastEncounterDischarge:
        return """Last encounter was discharge so Patient should probably be inactivated | """    

def filterAndSortToRelevantData(completeDataFrame, patientID):
    singlePatient_df = completeDataFrame[completeDataFrame['PatientID'] == patientID].copy()
    singlePatient_df = singlePatient_df[singlePatient_df['Summarized Encounter Type'].map(lambda encType: True if encType == 'Treatment Plan' or encType == 'Transfer Note' or encType == 'Discharge Note' else False)].copy()
    singlePatient_df = singlePatient_df.sort_values(by="Review Date", ascending=True)
    singlePatient_df = singlePatient_df.reset_index(drop=True)
    return singlePatient_df


def isEmptyDataFrame(singlePatient_df):     
    if singlePatient_df.shape[0] == 0:
        return True
    else:
        return False
    
def isNeedingToBeInactivated(is_ActiveInInSync, is_lastEncounterDischarge):
    if is_ActiveInInSync and is_lastEncounterDischarge:
        return True  
    else:
        return False

def isTransferNote(singlePatient_df, currentEncounterIndexNum):
    if singlePatient_df[currentEncounterIndexNum, 'Summarized Encounter Type'] == 'Transfer Note':
        return True
    else:
        return False

def hasNextPlan(singlePatient_df, currentEncounterIndexNum):
    # and see if there are any treatment plans after current treatment plan
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if 'Treatment Plan' in singlePatient_df[singlePatient_df['EncounterDate'] > currentTreatmentPlanDate]["Summarized Encounter Type"].unique():
        return True
    else:
        return False

def getLocationOfNextPlan(singlePatient_df, currentEncounterIndexNum):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    nextPlanLocation = singlePatient_df[(singlePatient_df['EncounterDate'] > currentTreatmentPlanDate) & (singlePatient_df["Summarized Encounter Type"] == 'Treatment Plan') ].index[0]
    return nextPlanLocation

def isTransferedNext(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan:
        # see if there are any transfer notes between current plan and next plan
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        if 'Transfer Note' in singlePatient_df[(singlePatient_df['EncounterDate'] > currentTreatmentPlanDate) & ((singlePatient_df['EncounterDate'] < nextTreatmentPlanDate))]["Summarized Encounter Type"].unique():
            return True
        else:
            return False
    else:
        if 'Transfer Note' in singlePatient_df[singlePatient_df['EncounterDate'] >= currentTreatmentPlanDate]["Summarized Encounter Type"].unique():
            return True
        else:
            return False
    
def isDischargedNext(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan: # see if there are any transfer notes between current plan and next plan
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        if 'Discharge Note' in singlePatient_df[(singlePatient_df['EncounterDate'] >= currentTreatmentPlanDate) & 
                                           ((singlePatient_df['EncounterDate'] < nextTreatmentPlanDate))]["Summarized Encounter Type"].unique():
            return True
        else:
            return False
    else: # see if there is a 
        if 'Discharge Note' in singlePatient_df[singlePatient_df['EncounterDate'] >= currentTreatmentPlanDate]["Summarized Encounter Type"].unique():
            return True
        else:
            return False    
    
def isNextPlanClosedLate(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation):
    if singlePatient_df.loc[nextPlanLocation, "EndDate"]!= pd.NaT and singlePatient_df.loc[nextPlanLocation, "EndDate"] > singlePatient_df.loc[currentEncounterIndexNum, "Review Date"]:
        return True
    else:
        return False

def isCurrentPlanSigned(singlePatient_df, currentEncounterIndexNum):
    if str(singlePatient_df.loc[currentEncounterIndexNum, "closedByFullName"]) =='None':
        return False
    else:
        return True

def getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan:
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        return (nextTreatmentPlanDate-singlePatient_df.loc[currentEncounterIndexNum, "EncounterDate"]).days
    else:
        daysAfterTransfer = (datetime.now()-currentTreatmentPlanDate).days
        return daysAfterTransfer

In [8]:
try:
    relevantInfodf = treatmentPlanEncounters.merge(patientDetails, how='left', on='PatientID')
    relevantInfodf = relevantInfodf[relevantInfodf['MRNNumber'].notnull()].copy()
    logger.info(f"Successfully merged patient details with treatment plan info.")
except Exception as e:
    logger.error(f"Failed to merge patient details with treatment plan info.", exc_info=True) 

## Calculate Review Date

In [9]:
try:
    relevantInfodf['Summarized Encounter Type'] = relevantInfodf['EncounterType'].map(lambda type: 
                                                              "Treatment Plan" if "treatment" in type.lower() else 
                                                               ("Initial Eval" if 'evaluation' in type.lower() else 
                                                                ("Transfer Note" if 'transfer' in type.lower() else 
                                                                 ("Discharge Note" if 'discharge' in type.lower() else "Something else"))))
    relevantInfodf['Review Date'] = relevantInfodf.apply(lambda row: row['EncounterDate'] + relativedelta(months=+6) - timedelta(days=1), axis=1)
    relevantInfodf['Days to Review Date'] = relevantInfodf['Review Date'].map(lambda reviewDate:  reviewDate.date()) - date.today()
    logger.info(f"Successfully calculated review date.")
except Exception as e:
    logger.error(f"Failed to calculate review date.", exc_info=True) 

## Determine Treatment Plan Status

In [10]:
try:
    allPatientDf = pd.DataFrame()
    for patientID in relevantInfodf['PatientID'].unique()[0:]:
        singlePatient_df = filterAndSortToRelevantData(completeDataFrame=relevantInfodf, patientID=patientID)
        if isEmptyDataFrame(singlePatient_df): #if patient has initial eval with no initial treatment plan
            emptySinglePatient_df = relevantInfodf[(relevantInfodf['PatientID'] == patientID)].copy()

            missingInitialTreatmentPlan = f"Missing Initial Treatment Plan"
            emptySinglePatient_df["missingInitialTreatmentPlan"] = find_missing_initial_treatment_plan(singlePatient_df)
            emptySinglePatient_df["treatmentPlanStatus"] = missingInitialTreatmentPlan

            allPatientDf = pd.concat([allPatientDf,emptySinglePatient_df])
            continue
        else:
            patientOverallStatus = ""

            missingInitialTreatmentPlan = find_missing_initial_treatment_plan(singlePatient_df)
            singlePatient_df['missingInitialTreatmentPlan'] = missingInitialTreatmentPlan        
            singlePatient_df["treatmentPlanStatus"] = missingInitialTreatmentPlan

            is_ActiveInInSync = singlePatient_df.loc[0, 'is_ActiveInInSync']
            is_lastEncounterDischarge = singlePatient_df.loc[0, 'is_lastEncounterDischarge']

            is_NeedToBeInactivated = isNeedingToBeInactivated(is_ActiveInInSync, is_lastEncounterDischarge)
            singlePatient_df['is_NeedToBeInactivated'] = is_NeedToBeInactivated

            for currentEncounterIndexNum in range(singlePatient_df.shape[0]):
                treatmentPlanStatus = patientOverallStatus
                currentEncounterType = singlePatient_df.loc[currentEncounterIndexNum, 'Summarized Encounter Type']
                daysToReviewDate = singlePatient_df.loc[currentEncounterIndexNum, 'Days to Review Date'].days

                has_NextPlan = hasNextPlan(singlePatient_df, currentEncounterIndexNum)
                singlePatient_df.loc[currentEncounterIndexNum, "has_NextPlan"] = has_NextPlan

                is_CurrentPlanSigned = isCurrentPlanSigned(singlePatient_df, currentEncounterIndexNum)
                singlePatient_df.loc[currentEncounterIndexNum, "is_CurrentPlanSigned"] = is_CurrentPlanSigned

                if not is_CurrentPlanSigned:
                    treatmentPlanStatus += "Plan Still needs to be signed | "       

                if is_NeedToBeInactivated:
                    treatmentPlanStatus += "Last encounter was discharge so Patient should probably be inactivated | "
                if has_NextPlan:
                    next_PlanLocation = getLocationOfNextPlan(singlePatient_df, currentEncounterIndexNum)
                    singlePatient_df.loc[currentEncounterIndexNum, "next_PlanLocation"] = next_PlanLocation

                    is_TransferedNext = isTransferedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                    singlePatient_df.loc[currentEncounterIndexNum, "is_TransferedNext"] = is_TransferedNext

                    is_DischargedNext = isDischargedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                    singlePatient_df.loc[currentEncounterIndexNum, "is_DischargedNext"] = is_DischargedNext
                    if (is_TransferedNext and not is_DischargedNext) and is_CurrentPlanSigned and has_NextPlan:
                        get_DaysAfterTransfer = getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                        treatmentPlanStatus += f"Old Plan is Good | Patient was Transfered and they completed the follow up plan {get_DaysAfterTransfer} days after."
                    elif (is_DischargedNext and not is_TransferedNext) and is_CurrentPlanSigned and has_NextPlan:
                        treatmentPlanStatus += f"Patient was Discharged"
                    elif not is_TransferedNext and not is_DischargedNext and is_CurrentPlanSigned and has_NextPlan:
                        treatmentPlanStatus += "Everything Seems Good"

                    is_NextPlanClosedLate = isNextPlanClosedLate(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                    singlePatient_df.loc[currentEncounterIndexNum, "is_NextPlanClosedLate"] = is_NextPlanClosedLate

                else: # if there is no next plan 
                    is_TransferedNext = isTransferedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                    singlePatient_df.loc[currentEncounterIndexNum, "is_TransferedNext"] = is_TransferedNext

                    is_DischargedNext = isDischargedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                    singlePatient_df.loc[currentEncounterIndexNum, "is_DischargedNext"] = is_DischargedNext

                    if not is_TransferedNext and not is_DischargedNext and daysToReviewDate > 0:
                        treatmentPlanStatus += f"Everything Seems Good. Still have {daysToReviewDate} days left."
                    elif not is_TransferedNext and not is_DischargedNext and daysToReviewDate < 0:
                        treatmentPlanStatus += f"Plan is {np.absolute(daysToReviewDate)} days overdue."
                    elif is_TransferedNext and not is_DischargedNext:
                        daysAfterTransfer = getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                        if daysAfterTransfer > 30 and currentEncounterType == "Transfer Note":
                            treatmentPlanStatus += f"Patient Transfered. Plan is {daysAfterTransfer-30} days overdue."
                        elif daysAfterTransfer > 30:
                            treatmentPlanStatus += f"Patient Transfered."
                        else:
                            treatmentPlanStatus += f"Patient Transfered. Plan is due in {30 - daysAfterTransfer} days."
                    elif (is_DischargedNext and not is_TransferedNext):
                        treatmentPlanStatus += f"Patient was Discharged"   
                singlePatient_df.loc[currentEncounterIndexNum, "treatmentPlanStatus"] = treatmentPlanStatus  
            allPatientDf = pd.concat([allPatientDf,singlePatient_df])
    logger.info(f"Successfully built all patients treatment plan details.")
except Exception as e:
    logger.error(f"Failed to build all patients treatment plan details.", exc_info=True) 

C:\Users\ylen\AppData\Local\Temp\ipykernel_1880\3867921235.py:12: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  allPatientDf = pd.concat([allPatientDf,emptySinglePatient_df])


In [11]:
allPatientDf[["has_NextPlan", 
              "is_CurrentPlanSigned",
              "next_PlanLocation",
              "is_TransferedNext",
              "is_DischargedNext",
              "is_NextPlanClosedLate"]] = allPatientDf[["has_NextPlan", 
                                                        "is_CurrentPlanSigned",
                                                        "next_PlanLocation",
                                                        "is_TransferedNext",
                                                        "is_DischargedNext",
                                                        "is_NextPlanClosedLate"]].fillna(False)   

In [12]:
list(allPatientDf['treatmentPlanStatus'])[2]

'Everything Seems Good'

In [13]:
# for column in allPatientDf.columns:
#     print(column)

In [14]:
try:
    allPatientDf['is_TreatmentPlanGood'] = allPatientDf['treatmentPlanStatus'].map(lambda status: True if 'Everything Seems Good' in str(status) else False)
    allPatientDf['is_TreatmentPlanOverdue'] = allPatientDf['treatmentPlanStatus'].map(lambda status: True if 'overdue' in str(status).lower() else False)
    logger.info(f"Successfully marked treatment plans that are all good.")
except Exception as e:
    logger.error(f"Failed to mark treatment plans that are all good.", exc_info=True) 

In [15]:
allPatientDf['treatmentPlanStatus'] = allPatientDf['treatmentPlanStatus'] + "| " + allPatientDf['seen_Recently']

In [16]:
# final_df[final_df['PatientID']==622214]
# final_df[final_df['PatientID']==622741]

In [17]:
# final_df[final_df['allTheInfoYouEverWanted'].map(lambda x: True if "Closed After Due Date" in x else False) ].groupby('ProviderName')['EncounterId'].count().sort_values(ascending=False).head(20)

In [18]:
# final_df[final_df['allTheInfoYouEverWanted']==""]

In [19]:
# Change to integer so SQL can recognize it
allPatientDf['Days to Review Date'] = allPatientDf['Days to Review Date'].dt.days

In [20]:
# filter to useful columns
allPatientDf = allPatientDf[["EncounterId",
                            "MRNNumber",
                            "DOB",
                            "PatientID",
                            "ProviderName",
                            "patient_fullName",
                            "FacilityName",
                            "Summarized Encounter Type",
                            "EncounterDate",
                            "EncounterEndDateTimeToolTip",
                            "EncounterStartDateTimeToolTip",
                            "EncounterType",
                            "EncounterOpenClosed",
                            "ProgramName",
                            "closedByFullName",
                            "finalEncounterType",
                            "finalEncounterDate",
                            "is_ActiveInInSync",
                            "is_lastEncounterDischarge",
                            "lastBillable_EncounterType",
                            "lastBillable_EncounterDate",
                            "lastBillable_ProgramName",
                            "lastBillable_FacilityName",
                            "seen_Recently",
                            "Review Date",
                            "Days to Review Date",
                            "is_NeedToBeInactivated",
                            "has_NextPlan",
                            "is_CurrentPlanSigned",
                            "next_PlanLocation",
                            "is_TransferedNext",
                            "is_DischargedNext",
                            "is_NextPlanClosedLate",
                            "treatmentPlanStatus",
                            "missingInitialTreatmentPlan",
                            "is_TreatmentPlanGood",
                            "is_TreatmentPlanOverdue",
                            "Admitted"]]

## For Presenting
- Replace all names with random ones for presenting

In [21]:
# randomNames = pd.read_excel('../data/Random Names.xlsx')

# final_df['patient_fullName'] = final_df['patient_fullName'].map(lambda name: list(randomNames['Names'].sample(1))[0])
# final_df['ProviderName'] = final_df['ProviderName'].map(lambda name: list(final_df['ProviderName'].sample(1))[0])

In [22]:
# use to transform data into
query = ", ".join([item.replace(" ", "_") + " " + str(allPatientDf[item].dtype) for item in allPatientDf.columns])
query = query.replace("object", "TEXT")
query = query.replace("datetime64[ns]", "TEXT")
query = query.replace("timedelta64[ns]", "TEXT")
query = query.replace("float64", "INTEGER")
query = query.replace("int64", "INTEGER")
query = query.replace("bool", "TEXT")
query

'EncounterId INTEGER, MRNNumber TEXT, DOB TEXT, PatientID INTEGER, ProviderName TEXT, patient_fullName TEXT, FacilityName TEXT, Summarized_Encounter_Type TEXT, EncounterDate TEXT, EncounterEndDateTimeToolTip TEXT, EncounterStartDateTimeToolTip TEXT, EncounterType TEXT, EncounterOpenClosed TEXT, ProgramName TEXT, closedByFullName TEXT, finalEncounterType TEXT, finalEncounterDate TEXT, is_ActiveInInSync INTEGER, is_lastEncounterDischarge INTEGER, lastBillable_EncounterType TEXT, lastBillable_EncounterDate TEXT, lastBillable_ProgramName TEXT, lastBillable_FacilityName TEXT, seen_Recently TEXT, Review_Date TEXT, Days_to_Review_Date INTEGER, is_NeedToBeInactivated TEXT, has_NextPlan TEXT, is_CurrentPlanSigned TEXT, next_PlanLocation TEXT, is_TransferedNext TEXT, is_DischargedNext TEXT, is_NextPlanClosedLate TEXT, treatmentPlanStatus TEXT, missingInitialTreatmentPlan TEXT, is_TreatmentPlanGood TEXT, is_TreatmentPlanOverdue TEXT, Admitted TEXT'

In [23]:
table_name= "ptTreatmentPlanDuenessSummaryCFTSS"
try:
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} ({query}) ''')
    conn.commit()
    allPatientDf.to_sql("ptTreatmentPlanDuenessSummaryCFTSS", conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed ptTreatmentPlanDuenessSummaryCFTSS to database.")
except Exception as e:
    logger.error(f"Failed to push ptTreatmentPlanDuenessSummaryCFTSS to database.", exc_info=True) 
    print(e)

In [24]:
conn.close()

## Old Code

In [25]:
# final_df[final_df['ProgramName'] =='Boro Park Clinics'].head(15).to_excel("../data/checkTreatmentPlanCode.xlsx", index=False)

In [26]:
# final_df[final_df['Patient Name'] == 'Braun, Chesky']

In [27]:
# final_df['Program']
# final_df['Status']
# final_df = final_df.merge(patientStatusProgramdf, on='MRN', how='inner')
# final_df

In [28]:
# final_df.columns

In [29]:
# final_df = final_df[['MRN', 'Patient Name_x', 'Encounter Type',
#                    'Encounter / Service Provider', "Encounter Facility", 'Visit Date', 'Signed By',
#                    'Signed Date', 'Co Sign Status', 'Co Signed By',
#                    'Summarized Encounter Type', 'Missing Initial Treatment Plan',
#                    'Review Date', 'Days to Review Date', 'allTheInfoYouEverWanted','Patient Status',
#                    'Date of Last Appointment', 'Total Appointments', 'Updated Status_x', "Last Seen_x"]].copy()

In [30]:
# writer = pd.ExcelWriter("Excel TreatmentPlan Data/TreatmentPlanforCompliance v5.xlsx", engine='xlsxwriter')

# # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
# # header and index and skip one row to allow us to insert a user defined
# # header.
# final_df.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

# # Get the xlsxwriter workbook and worksheet objects.
# workbook = writer.book
# worksheet = writer.sheets['Sheet1']

# # Get the dimensions of the dataframe.
# (max_row, max_col) = final_df.shape

# # Create a list of column headers, to use in add_table().
# column_settings = []
# for header in final_df.columns:
#     column_settings.append({'header': header})

# # Add the table.
# worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

# # Make the columns wider for clarity.
# worksheet.set_column(0, max_col - 1, 12)

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()
# writer.close()